In [1]:
%reload_ext autoreload
%autoreload 2


In [3]:
import pandas as pd
from taxi_demand_predictor.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,rides_previous_662_hour,rides_previous_661_hour,rides_previous_660_hour,rides_previous_659_hour,rides_previous_658_hour,rides_previous_657_hour,rides_previous_656_hour,rides_previous_655_hour,rides_previous_654_hour,rides_previous_653_hour,rides_previous_652_hour,rides_previous_651_hour,rides_previous_650_hour,rides_previous_649_hour,rides_previous_648_hour,rides_previous_647_hour,rides_previous_646_hour,rides_previous_645_hour,rides_previous_644_hour,rides_previous_643_hour,rides_previous_642_hour,rides_previous_641_hour,rides_previous_640_hour,rides_previous_639_hour,rides_previous_638_hour,rides_previous_637_hour,rides_previous_636_hour,rides_previous_635_hour,rides_previous_634_hour,rides_previous_633_hour,...,rides_previous_37_hour,rides_previous_36_hour,rides_previous_35_hour,rides_previous_34_hour,rides_previous_33_hour,rides_previous_32_hour,rides_previous_31_hour,rides_previous_30_hour,rides_previous_29_hour,rides_previous_28_hour,rides_previous_27_hour,rides_previous_26_hour,rides_previous_25_hour,rides_previous_24_hour,rides_previous_23_hour,rides_previous_22_hour,rides_previous_21_hour,rides_previous_20_hour,rides_previous_19_hour,rides_previous_18_hour,rides_previous_17_hour,rides_previous_16_hour,rides_previous_15_hour,rides_previous_14_hour,rides_previous_13_hour,rides_previous_12_hour,rides_previous_11_hour,rides_previous_10_hour,rides_previous_9_hour,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,1.0,2.0,1.0,5.0,5.0,13.0,3.0,3.0,1.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,0.0,1.0,1.0,3.0,3.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,0.0,1.0,1.0,3.0,3.0,0.0,9.0,5.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,2.0,...,0.0,0.0,1.0,3.0,3.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-01-30,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,2.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,2.0,1.0,4.0,2.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2022-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,2.0,1.0,2.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,...,1.0,2.0,1.0,4.0,2.0,2.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2022-02-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73135,3.0,4.0,1.0,1.0,1.0,0.0,1.0,2.0,2.0,3.0,3.0,6.0,4.0,0.0,3.0,4.0,0.0,2.0,3.0,0.0,13.0,1.0,7.0,4.0,9.0,4.0,3.0,3.0,2.0,4.0,2.0,6.0,1.0,3.0,0.0,4.0,3.0,

In [4]:
from datetime import datetime
from taxi_demand_predictor.data_split import train_test_split

# Split the data into train and test sets
X_train, y_train, X_test, y_test = train_test_split(df, cutoff_date=datetime(2022, 6, 1, 0, 0, 0), target_column_name='target_rides_next_hour')
print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32595, 674)
y_train.shape=(32595,)
X_test.shape=(40545, 674)
y_test.shape=(40545,)


In [5]:
import xgboost as xgb

In [8]:
# Only past rides for predictions
# So only get the columns that start with 'rides_'
# as they are the historical features
past_rides_columns = [c for c in X_train.columns if c.startswith('rides_')]
# Only take the columns that start with 'rides_'
X_train_only_numeric = X_train[past_rides_columns]

In [9]:
model = xgb.XGBRegressor()
# ONLY use historical data to train the model
model.fit(X_train_only_numeric, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [10]:
X_test_only_numeric = X_test[past_rides_columns]
predictions = model.predict(X_test_only_numeric)
predictions

array([0.06153813, 0.06668109, 0.31324342, ..., 7.9160233 , 5.8392587 ,
       4.5639987 ], dtype=float32)

In [11]:
from sklearn.metrics import mean_absolute_error
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=2.5616
